# Interpolación Kriging

In [ ]:
# Script for automating interpolations using the Kriging method of ArcGIS Pro
import arcpy

gdb_path = r"C:\Users\jhonatan quinones\....\MyProject.gdb"  # Replace path to the Geodatabase 
polygon_feature = "polygon" # Name of the feature class containing polygon to use as mask (Must be inside the Geodatabase)
point_feature = "points"  # Name of the feature class containing information to be interpolated (must be inside the Geodatabase)
pixel_size = 20  # Pixel size in meters (adjust according to coordinate system)

polygon_desc = arcpy.Describe(f"{gdb_path}\\{polygon_feature}") #To obtain the extension of the polygon
polygon_extent = polygon_desc.extent
extent_str = f"{polygon_extent.XMin} {polygon_extent.YMin} {polygon_extent.XMax} {polygon_extent.YMax}"

# Verify if the geodatabase and the feature class exist before continuing
if not arcpy.Exists(gdb_path):
    print("Error: Unable to access the geodatabase.")
elif not arcpy.Exists(f"{gdb_path}\\{point_feature}"):
    print(f"Error: The Feature class '{point_feature}' doesn't exist in the geodatabase.")
elif not arcpy.Exists(f"{gdb_path}\\{polygon_feature}"):
    print(F"Error: The Feature class '{polygon_feature}' doesn't exist in the geodatabase.")
else:
    with arcpy.EnvManager(extent=extent_str,mask=polygon_feature, scratchWorkspace=gdb_path):
        arcpy.env.overwriteOutput = True

        # Obtain the variables to interpolate
        variables = [f.name for f in arcpy.ListFields(point_feature)][6:8]  ### ! EDIT TO SELECT THE CORRECT VARIABLES (first column = 0)
        print(f"Se interpolarán las siguiente variables: {variables}")
        
        successful = 0  
        failed = 0
        aprx = arcpy.mp.ArcGISProject("CURRENT")  
        m = aprx.activeMap  

        for var in variables:
            print(f"Proccesing field {var}...")
            output_raster = f"{gdb_path}\\kriging_{var}"

            try:
                out_surface_raster = arcpy.sa.Kriging(
                    in_point_features = point_feature,
                    z_field = var,
                    kriging_model = "Spherical # # # #", # Replace by the desired model with this format “ModelType Nugget Sill Range Power”.
                    cell_size = pixel_size, 
                    search_radius = "VARIABLE 12",  # Replace by the desired search radius
                    out_variance_prediction_raster=None
                ).save(output_raster)
                if arcpy.Exists(output_raster):
                    raster_layer = m.addDataFromPath(output_raster)
                    if raster_layer:
                        raster_layer.name = f"kriging_{var}"                   
                    successful += 1
                    print(f"   Interpolation completed for {var}")
                else:
                    failed += 1
                    print(f"   Error: Raster {output_raster} was not successfully created.")

            except Exception as e:
                failed += 1
                print(f"   Error: An exception occurred with {var} - {e}")

        print(f"Kriging interpolation completed successfully for {successful} variables and failed for {failed} variables.")


# Interpolación Radial Basis Functions

In [ ]:
# Script for automating interpolations using the Radial Basis Functions method of ArcGIS Pro
import arcpy

gdb_path = r"C:\Users\jhonatan quinones\....\MyProject.gdb"  # Replace path to the Geodatabase 
polygon_feature = "polygon" # Name of the feature class containing polygon to use as mask (Must be inside the Geodatabase)
point_feature = "points"  # Name of the feature class containing information to be interpolated (must be inside the Geodatabase)
pixel_size = 20  # Pixel size in meters (adjust according to coordinate system)

polygon_desc = arcpy.Describe(f"{gdb_path}\\{polygon_feature}") #To obtain the extension of the polygon
polygon_extent = polygon_desc.extent
extent_str = f"{polygon_extent.XMin} {polygon_extent.YMin} {polygon_extent.XMax} {polygon_extent.YMax}"

# Verify if the geodatabase and the feature class exist before continuing
if not arcpy.Exists(gdb_path):
    print("Error: Unable to access the geodatabase")
elif not arcpy.Exists(f"{gdb_path}\\{point_feature}"):
    print(f"Error: The Feature class '{point_feature}' doesn't exit in the geodatabase.")
elif not arcpy.Exists(f"{gdb_path}\\{polygon_feature}"):
    print(F"Error: The Feature class '{polygon_feature}' doesn't exit in the geodatabase.")
else:
    with arcpy.EnvManager(extent=extent_str,mask=polygon_feature, scratchWorkspace=gdb_path):
        arcpy.env.overwriteOutput = True

        # Obtain the variables to interpolate
        variables = [f.name for f in arcpy.ListFields(point_feature)][6:8]  ###! EDIT TO SELECT THE CORRECT VARIABLES (first column = 0)
        print(f"Se interpolarán las siguiente variables: {variables}")
        
        successful = 0  
        failed = 0
        aprx = arcpy.mp.ArcGISProject("CURRENT")  
        m = aprx.activeMap  

        for var in variables:
            print(f"Proccesing field {var}...")
            output_raster = f"{gdb_path}\\rbf_{var}"

            try:
                out_surface_raster = arcpy.ga.RadialBasisFunctions(
                    in_features = point_feature,
                    z_field = var,
                    out_ga_layer = None,
                    out_raster = output_raster,
                    cell_size = pixel_size, 
                    search_neighborhood = "NBRTYPE=Standard S_MAJOR=529,164328933507 S_MINOR=529,164328933507 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",  # Replace with desired values
                    radial_basis_functions = "COMPLETELY_REGULARIZED_SPLINE", #Replace with desired (“THIN_PLATE_SPLINE”, “MULTI_QUADRIC”, “INVERSE_MULTI_QUADRIC”)
                    small_scale_parameter = None
                ).save(output_raster)
                if arcpy.Exists(output_raster):
                    raster_layer = m.addDataFromPath(output_raster)
                    if raster_layer:
                        raster_layer.name = f"rdf_{var}"                   
                    successful += 1
                    print(f"   Interpolation completed for {var}")
                else:
                    failed += 1
                    print(f"   Error: Raster {output_raster} was not successfully created")

            except Exception as e:
                failed += 1
                print(f"   Error: An exception occurred with {var} - {e}")

        print(f"Radial Basis Functions interpolation completed successfully for {successful} variables and failed for {failed} variables.)
